In [10]:
import random as rd
import astropy.io.fits as apf
import batman
import eleanor
import emcee
import getpass
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.backends.backend_pdf import PdfPages
from multiprocessing import Pool
from multiprocessing import Process
import multiprocessing
import numpy as np
import os
import pandas as pd
import shutil
import time as tm 
from astroquery.mast import Catalogs
import astropy.units as units
from astropy.wcs import WCS
import math
import astropy.io.fits as apf
from astropy.stats import sigma_clip
import warnings
warnings.filterwarnings("ignore")
from wotan import flatten
import lightkurve as lk
import corner
import numpy as np
import juliet
import matplotlib.gridspec as gridspec
import mpl_axes_aligner

from astropy.timeseries import BoxLeastSquares
from ldtk import LDPSetCreator, BoxcarFilter, TabulatedFilter, SVOFilter
from ldtk.filters import tess, sdss_z
from astroquery import svo_fps

import mr_forecast as mr
import numba
from math import floor
from functools import partial


from transitleastsquares import (
    transitleastsquares,
    cleaned_array,
    catalog_info,
    transit_mask
    )


import itertools

from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))



import deep_transit as dt
import copy

import sys

In [11]:
TICIDs = [150428135, 259377017, 235678745, 467179528, 284441182, 36724087, 441798995, 219195044]


In [12]:
from tqdm.auto import tqdm

import ntpath

LDC_for_quadratic = pd.read_csv('./LDC_files/table15.dat', 
                                header = None, 
                                sep="\s+", index_col=None,
                               names = ['logg', 'Teff', 'z','L/HP', 'aLSM', 'bLSM',
                                       'aFCM', 'bFCM', 'SQRT(CHI2)', 'qsr', 'PC'])

mdwarf_LDC_for_quadratic = LDC_for_quadratic[LDC_for_quadratic['Teff']<4300]
                  



In [13]:
def transit_mask(t, period, duration, T0):
    
    # Works with numba, but is not faster
    mask = np.abs((t - T0 + 0.5 * period) % period - 0.5 * period) < 0.5 * duration
    return mask


def match_logg_and_teff_for_LDC(df):
    
    # Written by Mallory Harris

    # Description: uses logg and effective temp to calculate quadratic limb darkening parameters based on given  csv
    
    # Arguments : df = panda dataframe of TIC parameters


    # Return    : df = panda dataframe with updated quadratic limb darkening parameters

    a = []
    b = []
#     print('a')
    bar_format = "{desc}{percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} targets | {elapsed}<{remaining}"
    pbar = tqdm(total=len(df), smoothing=0.3,  position=1, leave=True, bar_format=bar_format)
    for i in range(len(df)):
#         print(i)

        Teff = np.float128(df['Teff'])[i]
        logg =  np.float128(df['logg'])[i]
        pbar.update(1)

        try:
            int(Teff)
            mdwarf_Teff =mdwarf_LDC_for_quadratic[mdwarf_LDC_for_quadratic['Teff'] == np.median(mdwarf_LDC_for_quadratic.iloc[(mdwarf_LDC_for_quadratic['Teff'].astype('float128')-Teff+0.01).abs().argsort()[:8]].reset_index(drop=True)['Teff'])]
            if i%100 == 0:
    #                 print(Teff, set(mdwarf_Teff['Teff']))
                if len(set(mdwarf_Teff['Teff']))>1:
                       print(mdwarf_Teff)
            if not abs(logg)>=0.:
                logg = np.median(mdwarf_Teff['logg'])
    #         print(mdwarf_Teff.iloc[(mdwarf_Teff['logg'].astype('float128')-logg-0.00000001).abs().argsort()].iloc[0][['aLSM', 'bLSM']])
            aLSM, bLSM =  mdwarf_Teff.iloc[(mdwarf_Teff['logg'].astype('float128')-logg-0.00000001).abs().argsort()].iloc[0][['aLSM', 'bLSM']]
#             print(aLSM, bLSM)
            a.append(aLSM)
            b.append(bLSM)  

                    
        except:
            a.append(np.nan)
            b.append(np.nan)
    pbar.close()

    df['aLSM'] = a
    df['bLSM'] = b
    return df
    

In [14]:
TICIDs = [261257684, 150428135, 259377017, 235678745, 467179528, 284441182, 36724087, 441798995, 219195044]


In [15]:
ctlfile = './Total_Mdwarf_files/final_mdwarf_params_new.csv'
mdwarfs = pd.read_csv(ctlfile, iterator =True, chunksize = 100000, index_col=None)

mdwarfs_known = pd.concat(
    [chunk[chunk['TICID'].astype(int).isin(TICIDs)] 
    for chunk in mdwarfs]
).reset_index(drop=True)

In [29]:
def mkdir_if_doesnt_exist(outdir, str_new_dir_name):
    # Written by Mallory Harris

    # Description: creates new directory to save data to if it does not already exist

    # Arguments : outdir             = existing directory in which new directory will be located
    #             str_new_dir_name   = string of the new subdirectory of outdir's name


    if os.path.exists(outdir+str_new_dir_name)==False:
        new_outdir = os.path.join(outdir, str_new_dir_name)
        os.mkdir(new_outdir)


def fetch(ticid, pipeline, exptime, sector_num): 
    # Written by Mallory Harris

    # Description: uses lightkurve to download TESS fits file based on sector number, source pipeline, exposure time and star id

    # Arguments : ticid       = TESS Input Catalog identification number of star
    #             sector_num  = sector from which to download data

    #             pipeline    = source pipeline from which to get data (options are 'QLP', 'SPOC', 'TESS-SPOC')
    #             exptime     = exposure length of observations (2 or 30 minutes, depending on pipeline) units of seconds
    #                           'QLP', 'TESS-SPOC', 'TGLC' = 30 minute (1800s, PM), 10 minute (600s, EM1), 300s (EM2)
    #                           'SPOC'             = 2  minutte (120 s) exptime

    Path = './known_Mdwarfs_data'
    good_authors = [pipeline]
    if pipeline == 'SPOC':
        Path =  Path+'/PS_data/tic_'+str(ticid)
        small_dir_path_lst = glob.glob(Path+"/mastDownload/TESS/*"+str(ticid).zfill(16)+"*")
    else:
        Path =  Path+'/FFI_data/tic_'+str(ticid)
#         print(Path)
        small_dir_path_lst = glob.glob(Path+"/HLSP/*"+str(ticid).zfill(16)+"*")

        if pipeline == 'QLP':
            small_dir_path_lst = [i for i in small_dir_path_lst if 'ffi' in i]
        elif pipeline == 'TESS-SPOC':
            small_dir_path_lst = [i for i in small_dir_path_lst if 'tess-spoc' in i]
        elif pipeline == 'TGLC':
            small_dir_path_lst = [i for i in small_dir_path_lst if 'gaiaid' in i]

    for i in small_dir_path_lst:
        file = glob.glob(i+'/*.fits')
        print('there is something already in the folder:', len(file))
        if len(file)>0:
            os.remove(file[0])
        print('it was removed')
        
    search = lk.search_lightcurve('TIC '+str(ticid), mission='TESS', exptime =exptime)
#     print(search)
    try:
        result = all(elem in list(search.author)  for elem in good_authors)
    except:
        print(search)
        result = False

    if result == True:
        search = search[search.author == pipeline]
        if sector_num!= None:
            search.download(quality_bitmask='default', download_dir = Path)
        else:
            
            search.download_all(quality_bitmask='default', download_dir = Path)

        print('done downloading')
    else:
        print('No '+pipeline+' obervations')

def get_lcf_files(df, pipeline, last_hung_ticid = 0, sector_num = False):
    # Written by Mallory Harris

    # Description: takes dataframe of TICIDs observed in a given sector (identified by a column in df, explained below)
    #              to download light curves of that sector for each ticid from given pipeline 
            

    # Arguments : df              = panda dataframe. Must include at least one column of "TICID" with TESS identification number
    #                               and one column denoting whether or not observations were taken in the named sector.
    #                               example: if targets are observed in sector 20, their value in the column "S20" will be >0
    #             sector_num      = sector from which to download data
    #             pipeline        = source pipeline from which to get data (options are 'QLP', 'SPOC', 'TESS-SPOC')
    #             last_hung_ticid = if, for any reason, this function is stopped, one can enter that last downloaded TICID to avoid re-downloading data

    if pipeline == 'SPOC':
        exptime = 'short'
    else:
        exptime = 'long'
    if sector_num!=False:
        new_df = df[df['S'+str(sector_num)]>0.].reset_index(drop = True)
        print('sector '+str(sector_num), 'num targets', len(new_df))

    else:
        new_df = df.copy()
        sector_num = None
    indx = 0

    if last_hung_ticid != 0:
        indx = new_df[new_df['TICID'] == last_hung_ticid].index[0]
    print('starting index: ', indx)
    r = indx
    sector_PS_targets = new_df[indx:]
    #start_time = time.time()
    masteroutdir = './known_Mdwarfs_data/'
    
    if pipeline == 'SPOC':
        mkdir_if_doesnt_exist(masteroutdir, 'PS_data/')
        masteroutdir = masteroutdir + 'PS_data/'
    else:
        mkdir_if_doesnt_exist(masteroutdir, 'FFI_data/')
        masteroutdir = masteroutdir + 'FFI_data/'

    
    for i in sector_PS_targets['TICID']:
        print(r)
        r +=1

        ticid = int(i)
        
        print('TIC '+str(ticid))
        #print(time.localtime())
        while 1<2:
            proc = Process(target=fetch, args=(ticid, sector_num, pipeline, exptime))
            proc.start()
            proc.join(timeout = 45)
            proc.terminate()
            if proc.exitcode is None:
                print('timeout')
                continue
            else:
                print('success!')
                break
    print('All done!')


In [31]:
mdwarfs_known

,TICID,RA,DEC,Vmag,Jmag,Tmag,Teff,eTeff_x,Rad,eRad,...,S54,S55,eTeff_y,logg,elogg,M/H,eM/H,GAIA,aLSM,bLSM
0,261257684.0,89.372315,-83.130204,12.588,9.607,10.8460,3751.0,157.0,0.532439,0.015794,...,0.0,0.0,157.0,4.70914,0.009127,NaN,NaN,4.620844e+18,0.2581,0.3609
1,150428135.0,97.096787,-65.579312,13.151,9.469,10.9102,3494.0,157.0,0.419457,0.012573,...,0.0,0.0,157.0,4.80892,0.004651,NaN,NaN,5.284518e+18,0.1604,0.4325
2,219195044.0,92.391740,-53.823484,13.311,10.241,11.5436,3731.0,157.0,0.473141,0.014147,...,0.0,0.0,157.0,4.76101,0.007200,NaN,NaN,5.500474e+18,0.1737,0.4118
3,259377017.0,68.415501,-51.956232,12.603,9.099,10.4981,3532.0,157.0,0.374358,0.011496,...,0.0,0.0,157.0,4.85053,0.002090,NaN,NaN,4.781196e+18,0.1604,0.4325
4,36724087.0,154.646405,-11.716734,13.140,9.007,10.5848,3331.0,157.0,0.374139,0.011184,...,0.0,0.0,157.0,4.85074,0.001634,NaN,NaN,3.767282e+18,0.1529,0.4604
5,235678745.0,285.633044,75.418605,13.190,9.797,11.0788,3746.0,157.0,0.463205,0.013592,...,10.0,10.0,157.0,4.76979,0.006479,NaN,NaN,2.268372e+18,0.1737,0.4118
6,441798995.0,264.147791,77.126122,13.476,10.765,11.9521,3872.0,157.0,0.548484,0.016161,...,10.0,10.0,157.0,4.69525,0.009462,NaN,NaN,1.657159e+18,0.2946,0.3316
7,284441182.0,10.088709,61.213626,13.459,10.154,11.4752,3624.0,157.0,0.490120,0.014531,...,0.0,0.0,157.0,4.74607,0.007723,NaN,NaN,4.272241e+17,0.2125,0.3984
8,467179528.0,197.998168,65.833807,12.941,9.706,11.0402,3618.0,157.0,0.435666,0.012939,...,0.0,0.0,157.0,4.79432,0.005376,NaN,NaN,1.678074e+18,0.1671,0.4201


In [30]:
fetch(261257684, 'SPOC', 'short', None)
# 

there is something already in the folder: 0
it was removed
there is something already in the folder: 0
it was removed
there is something already in the folder: 0
it was removed
there is something already in the folder: 0
it was removed
done downloading


In [32]:
def find_breaks(time, val = 27.):
    time = time[np.argsort(time)] 
    t   = np.diff(time)
    inds = np.where( t>val)[0]
    return inds + 1

# def sort_data_by_times(time, flux):
#     return time[np.argsort(time)], flux[np.argsort(time)]


def breaking_up_data(time, break_val = 27.):
    brk = np.append(np.append([0], find_breaks(time, break_val)), [len(time)])
    
    
    indexes = []
    for i in range(len(brk)-1):
        r = np.arange(brk[i],brk[i+1], 1)

        cadence_6hrs = np.nanmin(np.diff(new_time[r]))/24/4
    
        if len(time[r])>cadence_6hrs:
            indexes.append(r)
    return indexes


In [33]:
def running_median(data, kernel=300):
    """Returns sliding median of width 'kernel' and same length as data """
    idx = np.arange(kernel) + np.arange(len(data) - kernel + 1)[:, None]
    idx = idx.astype(np.int)  # needed if oversampling_factor is not int
    med = np.median(data[idx], axis=1)

    # Append the first/last value at the beginning/end to match the length of
    # data and returned median
    first_values = med[0]
    last_values = med[-1]
    missing_values = len(data) - len(med)
    values_front = int(missing_values * 0.5)
    values_end = missing_values - values_front
    med = np.append(np.full(values_front, first_values), med)
    med = np.append(med, np.full(values_end, last_values))
    return med

In [34]:
300*5/60/60/24

0.017361111111111112

In [35]:
0.005*24*60

7.199999999999999

In [36]:
# import copy

# model_colors = ['#FF930F', '#71Fd8FF']
def using_BLS_to_find_periodic_signals(time, flux, u, verbose = True, show_progress_info = True, 
                                       intransit = [], periods = [], T0 = [], Tdur = [], 
                                       depths = [], first=True):
    
    max_per = min(max(time)-min(time), 100.)
    if first == True:
        intransit = np.full(len(time), False)
        periods = []
        T0 = []
        Tdur = []
        depths = []

    time_new = time[~intransit]
    flux_new = flux[~intransit]
    if len(time_new)>0:
        start = tm.time()

        durations = np.linspace(0.02, 0.5, 75)
        model     = BoxLeastSquares(time_new, flux_new)
        results   = model.autopower(durations, objective='snr', maximum_period=max_per)
        
        my_median = running_median(results.power )
        
        results['power_final'] = results.power - my_median
        
        index = np.argmax(results.power_final)
        print('results ', results)
        period    = results.period[index]
        print('period', period, 'index ', index)
        print('sizes', len(results.transit_time), len(results.period))
#     print('params', periods, T0, Tdur, depths)
        end = tm.time()
        print('planet run: BLS time ', (end-start)/60, ' minutes')

#         else:d
        if round(results.transit_time[index], 4) in [round(x, 4) for x in T0]:
        
            intransit = np.logical_or(intransit, transit_mask(time, results.period, 2*results.duration+(2*0.04), results.transit_time))
            print('mask', set(transit_mask(time, results.period, 2*results.duration+(2*0.04), results.transit_time)))
            return using_BLS_to_find_periodic_signals(time, flux, u, intransit=intransit, verbose = verbose,  periods = periods, T0 = T0, Tdur = Tdur, depths = depths, first = False)
        if verbose:
            
            print('duration picked: ', results.duration[index])
            plt.figure(figsize = (10, 10))

            ax = plt.gca()
            ax.set_facecolor('None')
            ax.axvline(period, alpha=1, lw=3.5,  color='#E4D7BA')
            plt.xlim(np.min(results.period), np.max(results.period))
            for n in range(2, 10):
                ax.axvline(n*period, alpha=0.25, lw=2, linestyle="dashed",  color='#E4D7BA')
                ax.axvline(period / n, alpha=0.25, lw=2, linestyle="dashed",  color='#E4D7BA')
            for per in periods:
                ax.axvline(per, alpha=0.6, lw=2, linestyle="dashed",  color='#E4D7BA')

#                         ax.axvline(results.period / n, alpha=0.4, lw=1, linestyle="dashed",  color='#B06200')
            plt.ylabel(r'SDE', fontsize = 40)
            plt.xlabel('Period (days)', fontsize = 40)
        
        
        
#             chi2 = 1 / results.power
#             SR = min(chi2) / chi2
#             SDE = (1 - np.mean(SR)) / np.std(SR)
#             SDE_power = SR - np.min(SR)      # shift down to touch 0
#             scale = SDE / np.max(SDE_power)  # scale factor to touch max=SDE
#             SDE_power = SDE_power * scale
            
            
            
            ax.plot(results.period, results.power, color = '#EB8300', lw=0.65)
            ax.plot(results.period, results.power_final, color = 'k', lw=0.65)
            ax.scatter(results.period, my_median, color = 'r')
            
#             plt.xlim(0, 100.);
            ax.xaxis.label.set_color('#BC9B54')        #setting up X-axis label color to yellow
            ax.yaxis.label.set_color('#BC9B54')          #setting up Y-axis label color to blue

            ax.tick_params(axis='x', colors='#BC9B54', labelsize = 35, width=3, length = 8)    #setting up X-axis tick color to red
            ax.tick_params(axis='y', colors='#BC9B54', labelsize = 35, width=3, length = 8)  #setting up Y-axis tick color to black

            ax.spines['left'].set_color(None)        # setting up Y-axis tick color to red
            ax.spines['bottom'].set_color(None)         #setting up above X-axis tick color to red
            ax.spines['top'].set_color(None)        # setting up Y-axis tick color to red
            ax.spines['right'].set_color(None)         #setting up above X-axis tick color to red




            t0 = results.transit_time[index]
            duration = results.duration[index]
            
            plt.show()
            plt.close()
            
            plt.figure(figsize = (10, 10))
            ax2 = plt.gca()
            ax2.xaxis.label.set_color('#BC9B54')        #setting up X-axis label color to yellow
            ax2.yaxis.label.set_color('#BC9B54')          #setting up Y-axis label color to blue

            ax2.tick_params(axis='x', colors='#BC9B54', labelsize = 35, width=3, length = 8)    #setting up X-axis tick color to red
            ax2.tick_params(axis='y', colors='#BC9B54', labelsize = 35, width=3, length = 8)  #setting up Y-axis tick color to black
            ax2.spines['top'].set_color(None)        # setting up Y-axis tick color to red
            ax2.spines['right'].set_color(None)         #setting up above X-axis tick color to red

            ax2.spines['left'].set_color(None)        # setting up Y-axis tick color to red
            ax2.spines['bottom'].set_color(None)         #setting up above X-axis tick color to red

            ax2.set_facecolor('None')

            x = ((time - t0 + 0.5*period) % period) -( 0.5*period)
            m = np.abs(x) < 0.5
            ax2.scatter(
                x[m],
                flux[m],
                color='#E4D7BA',
                s=10,
                alpha=0.8,
                zorder=2)

            x_new = np.linspace(-0.5, 0.5, 1000)
            f = model.model(x_new + t0, period, duration, t0)

            ax2.plot(x_new, f, color='#EB8300', lw = 6.5)
#             ax2.set_xlim(-0.5, 0.5)
            ax2.set_xlabel('Phase', color = '#BC9B54', fontsize = 40)
            ax2.set_ylabel('Relative Flux', color = '#BC9B54', fontsize = 40);
            plt.show()
            print('T0: ', results.transit_time[index], 'duration: ', results.duration[index], 'npoints_dur: ', np.ceil(results.duration[index]/30.))
        if not np.abs(np.diff(np.array(sorted(results.power_final))[[-1, -2]]))>2*np.nanstd(np.diff(results.power_final)) or len(time_new)==0 :

            print('done with multis!')
            print('per', periods)

            return np.array(periods), np.array(T0), np.array(Tdur), np.array(depths)
        else: 
            intransit = np.logical_or(intransit, transit_mask(time, results.period[index], 2*results.duration[index]+(2*0.04), results.transit_time[index]))
            
            depths.append(1-results.depth[index])
            periods.append(results.period[index])
            T0.append(results.transit_time[index])
            Tdur.append(results.duration[index])

            return using_BLS_to_find_periodic_signals(time, flux, u, intransit=intransit, verbose = verbose,  periods = periods, T0 = T0, Tdur = Tdur, depths = depths, first = False)



In [37]:
def get_catalog_info(ticid, df = False):
    if type(df) == bool:
        ctlfile = './Total_Mdwarf_files/final_mdwarf_params_new.csv'
        mdwarfs = pd.read_csv(ctlfile, iterator =True, chunksize = 100000, index_col=None, header = 0)
        df = pd.concat(
            [chunk[chunk['TICID'].astype(int) == ticid] 
            for chunk in mdwarfs]
        ).reset_index(drop=True)
    else:
        df = df[df['TICID']==ticid]
    return df[['aLSM', 'bLSM']].values[0].astype(float), float(df['Mass']), float(df['eMass']), float(df['eMass']), float(df['Rad']), float(df['eRad']), float(df['eRad'])




In [38]:


def running_periodic_search(files, ):
    total_time, total_flux = get_data(files)

    time = total_time[np.argsort(total_time)]
    flux = total_flux[np.argsort(total_time)]
    flux_err = np.full(len(flux), np.std(flux))

    
    brk = np.append(np.append([0], find_breaks(total_sorted_times)), [len(total_sorted_times)])

    indexes = breaking_up_data(new_time, brk)
    split_times    = [time[indx]     for indx in indexes]
    split_flux     = [flux[indx]     for indx in indexes]
    split_flux_err = [flux_err[indx] for indx in indexes]
    
    intransit = np.full(len(total_sorted_times), False)

    tf = []
    for mmm in range(len(split_times)):
        tf.append([split_times[mmm], split_fluxes[mmm]])
#         print('is this working?', len(tf[mmm]), len(tf[mmm][0]), len(tf[mmm][1]))


#     time, flux = new_binning_function_by_time(list(time), list(flux), 10.)
    print(len(time), len(flux))

    print('running periodic search')

    periods_multis, T0_multis, Tdur_multis, depth_multis = using_BLS_to_find_periodic_signals(time, flux, u = ab, verbose = verbose)
    
    for iii in range(len(periods_multis)):
        
        intransit = np.logical_or(intransit, transit_mask(total_sorted_times, periods_multis[iii], (2*Tdur_multis[iii])+0.08, T0_multis[iii]))
        
        
        


In [39]:
def running_mean(data, window_size):
    """Returns the moving average of an array `data`.
    Parameters
    ----------
    data : array of numbers
        The running mean will be computed on this data.
    window_size : int
        Window length used to compute the running mean.
    """
    cumsum = np.cumsum(np.insert(data, 0, 0))
    return (cumsum[window_size:] - cumsum[:-window_size]) / float(window_size)


def mask_outliers(time, flux, sigma = 4.5, sigma_lower=None, sigma_upper=None, **kwargs):
    outlier_mask = sigma_clip(data=flux,
                              sigma=sigma,
                              sigma_lower=sigma_lower,
                              sigma_upper=sigma_upper,
                              **kwargs).mask
    # Second, we return the masked light curve and optionally the mask itself
    return outlier_mask
    



def remove_outliers(time, flux, sigma = 4.5, sigma_lower=None, sigma_upper=None, **kwargs):
    outlier_mask = sigma_clip(data=flux,
                              sigma=sigma,
                              sigma_lower=sigma_lower,
                              sigma_upper=sigma_upper,
                              **kwargs).mask
    # Second, we return the masked light curve and optionally the mask itself
    return time[~outlier_mask], flux[~outlier_mask]
    
def flatten_lc(time, flux):
    flat_flux, flux_trend = flatten(
    time,                 # Array of time values
    flux,                 # Array of flux values
    method='biweight',
    window_length=0.9,    # The length of the filter window in units of ``time``
#     edge_cutoff=0.1,      # length (in units of time) to be cut off each edge.
#     break_tolerance=1,  # Split into segments at breaks longer than that
    return_trend=True,    # Return trend and flattened light curve
#     cval=5.0              # Tuning parameter for the robust estimators
    )
    return flat_flux, flux_trend

def cleaning_up_data(time, flux):
#     flux = flux/np.nanmedian(flux)
#     try: 
#         flattened_flux = flatten_lc(time, flux)

#     except Exception as error: 
#         flattened_flux = flux

    cleaned_time, cleaned_flux = remove_outliers(time, flux)
    return cleaned_time, cleaned_flux





def get_data(fits_files_lst_for_target):
    total_time = []
    total_flux = []
    total_flux_err = []


    for i in fits_files_lst_for_target:
#         sector = np.int64(i.split('/')[7].split('_')[4].split('-')[0][1:])
#         print('sector', sector)
#         if sector > 34:
#             print(hdulist)

        hdulist=apf.open(i) #fits time series

        tbdata = hdulist[1].data
#         print(tbdata)
        qual         = tbdata.QUALITY == 0

        time, flux = tbdata.TIME[qual], tbdata.SAP_FLUX[qual]
        
        print('init time', len(time))

        flux = flux/np.nanmedian(flux)
        flux, time = flux[~np.isnan(flux)], time[~np.isnan(flux)]

        cleaned_time, cleaned_flux = cleaning_up_data(time, flux)

        flat_flux, flat_trend = flatten_lc(cleaned_time, cleaned_flux)

        flat_flux_err = np.full(len(flat_flux), np.std(flat_flux))
        
        total_time.extend(cleaned_time)
        total_flux.extend(flat_flux)
        total_flux_err.extend(flat_flux_err)

#         plt.scatter(total_time, total_flux)
#         plt.show()BaseException
    print('total_time', len(total_time))
    return np.array(total_time), np.array(total_flux), np.array(total_flux_err)

    

In [40]:
def get_catalog_info(ticid, df = False):
    if type(df) == bool:
        ctlfile = './Total_Mdwarf_files/final_mdwarf_params_new.csv'
        mdwarfs = pd.read_csv(ctlfile, iterator =True, chunksize = 100000, index_col=None, header = 0)
        df = pd.concat(
            [chunk[chunk['TICID'].astype(int) == ticid] 
            for chunk in mdwarfs]
        ).reset_index(drop=True)
    else:
        df = df[df['TICID']==ticid]
    return df[['aLSM', 'bLSM']].values[0].astype(float), float(df['Mass']), float(df['eMass']), float(df['eMass']), float(df['Rad']), float(df['eRad']), float(df['eRad'])




In [41]:
ctlfile = './Total_Mdwarf_files/final_mdwarf_params.csv'
mdwarfs = pd.read_csv(ctlfile, iterator =True, chunksize = 100000, index_col=None)

mdwarfs_known = pd.concat(
    [chunk[chunk['TICID'].astype(int).isin(TICIDs)] 
    for chunk in mdwarfs]
).reset_index(drop=True)

In [42]:
mdwarfs_known =match_logg_and_teff_for_LDC(mdwarfs_known)

  0%|          | 0/9 targets | 00:00<?

In [43]:
mdwarfs_known

,TICID,RA,DEC,Vmag,Jmag,Tmag,Teff,eTeff_x,Rad,eRad,...,S54,S55,eTeff_y,logg,elogg,M/H,eM/H,GAIA,aLSM,bLSM
0,261257684.0,89.372315,-83.130204,12.588,9.607,10.8460,3751.0,157.0,0.532439,0.015794,...,0.0,0.0,157.0,4.70914,0.009127,NaN,NaN,4.620844e+18,0.2581,0.3609
1,150428135.0,97.096787,-65.579312,13.151,9.469,10.9102,3494.0,157.0,0.419457,0.012573,...,0.0,0.0,157.0,4.80892,0.004651,NaN,NaN,5.284518e+18,0.1604,0.4325
2,219195044.0,92.391740,-53.823484,13.311,10.241,11.5436,3731.0,157.0,0.473141,0.014147,...,0.0,0.0,157.0,4.76101,0.007200,NaN,NaN,5.500474e+18,0.1737,0.4118
3,259377017.0,68.415501,-51.956232,12.603,9.099,10.4981,3532.0,157.0,0.374358,0.011496,...,0.0,0.0,157.0,4.85053,0.002090,NaN,NaN,4.781196e+18,0.1604,0.4325
4,36724087.0,154.646405,-11.716734,13.140,9.007,10.5848,3331.0,157.0,0.374139,0.011184,...,0.0,0.0,157.0,4.85074,0.001634,NaN,NaN,3.767282e+18,0.1529,0.4604
5,235678745.0,285.633044,75.418605,13.190,9.797,11.0788,3746.0,157.0,0.463205,0.013592,...,10.0,10.0,157.0,4.76979,0.006479,NaN,NaN,2.268372e+18,0.1737,0.4118
6,441798995.0,264.147791,77.126122,13.476,10.765,11.9521,3872.0,157.0,0.548484,0.016161,...,10.0,10.0,157.0,4.69525,0.009462,NaN,NaN,1.657159e+18,0.2946,0.3316
7,284441182.0,10.088709,61.213626,13.459,10.154,11.4752,3624.0,157.0,0.490120,0.014531,...,0.0,0.0,157.0,4.74607,0.007723,NaN,NaN,4.272241e+17,0.2125,0.3984
8,467179528.0,197.998168,65.833807,12.941,9.706,11.0402,3618.0,157.0,0.435666,0.012939,...,0.0,0.0,157.0,4.79432,0.005376,NaN,NaN,1.678074e+18,0.1671,0.4201


In [72]:
model_path = './model_TESS.pth'


def make_LightKurveObject(time, flux, flux_err):
    """ Convert this object to a lightkurve.lightcurve.TessLightCurve object to use for Deep Transit.

    :return: Data in a lightkurve object
    :rtype: `class` lightkurve.lightcurve.TessLightCurve
    """
    lc = lk.TessLightCurve()
    lc.time = time
    lc.flux = flux
    lc.flux_err = flux_err
    return lc


def DT_analysis(time, flux, flux_err, DT_Quite=True, is_flat = True):
    """ 
    @author: D. Dragomir, P.Steimle


    Function for the Deep Transit analysis (Cui et al. 2021) that returns only single transit events

    """

    # create a new dataset for the while loop where transits can be masked out


    if DT_Quite == True:
        save_stdout = sys.stdout
        save_stderr = sys.stderr
        sys.stdout = open('.trash.txt', 'w')
        sys.stderr = open('.trash.txt', 'w')

        # do check for transits with DT
        DT_model = dt.DeepTransit(make_LightKurveObject(time, flux, flux_err),  is_flat=is_flat)
        bboxes = DT_model.transit_detection(model_path, confidence_threshold=0.6)

        sys.stdout = save_stdout
        sys.stderr = save_stderr

    else:
        # do check for transits with DT
        DT_model = dt.DeepTransit(make_LightKurveObject(time, flux, flux_err), is_flat=is_flat)
        bboxes = DT_model.transit_detection(model_path, confidence_threshold=0.6)

        # if only 1 or 0 boxes were found, break the loop and continue

    return bboxes


In [73]:
def new_binning_function_by_time(time, flux, time_size_of_bins): 
    # Written by Mallory Harris

    # Description: bins data in time, flux, and raw flux arrays as their mean value based on given time interval

    # Arguments : time              = array of time, in days
    #             flux              = array of flux 
    #             time_size_of_bins = bin size for time intervals, in minutes


    # Return    : list of binned time, flux and 
    time = np.array(time).byteswap().newbyteorder()  #changes how the arrays are represented in memory
    flux = np.array(flux).byteswap().newbyteorder() 

    interval = time_size_of_bins/60./24.#change units from minutes to days
    df = pd.DataFrame({'time': time, 'flux':flux})
    df = df.append(df)
    #print(df)
    numbins = np.array(list(range(int(np.ceil((max(time)-min(time))/interval))+1)))
    #print(df)
    bins = np.array([min(time)+x*interval for x in numbins])
    #print(bins)
    df['time_bins'] = pd.cut(df.time, bins)
    new_time = [x for x in df.groupby('time_bins').mean()['time'] if not math.isnan(x)] #finds the mean value of binned time
    new_flux = [x for x in df.groupby('time_bins').mean()['flux'] if not math.isnan(x)] #finds the mean value of binned flux
    #print(len(new_time), len(new_flux), len(new_raw))
    return np.array(new_time), np.array(new_flux)


In [46]:
def plot_lc_with_bboxes(lc_object, bboxes, ax=None, **kwargs):
    """
    Plot light curve with bounding boxes

    Parameters
    ----------
    lc_object : `~lightkurve.LightCurve` instance

    bboxes : list or np.ndarray
                Bounding boxes in shape (N, 5)

    ax : `~matplotlib.pyplot.axis` instance
                Axis to plot to. If None, create a new one.
    kwargs : dict
                Additional arguments to be passed to `matplotlib.pyplot.plot`

    Returns
    -------
    ax : `~matplotlib.pyplot.axis` instance
                The matplotlib axes object.
    """
    with plt.style.context('grayscale'):
        if ax is None:
            fig, ax = plt.subplots(1, figsize=(12, 26), constrained_layout=False)
            ax.plot(lc_object.time.value, lc_object.flux.value, color = '#996633', **kwargs)
        else:
            ax.plot(lc_object.time.value, lc_object.flux.value,color = '#996633', **kwargs)
        from matplotlib.patches import Rectangle
        from matplotlib.collections import PatchCollection
        recs = []
        for real_mask in bboxes:
            rec = Rectangle((real_mask[1] - real_mask[3] / 2, real_mask[2] - real_mask[4] / 2),
                            real_mask[3],
                            real_mask[4], fill=False, color='#CC9966')
            recs.append(rec)

            ax.text(
                real_mask[1] - real_mask[3] / 2,
                real_mask[2] + real_mask[4] / 2,
                s=f"{real_mask[0]:.2f}",
                color='white',
                verticalalignment="top",
                bbox=dict(alpha=0.75, color='#996633'),
                clip_on=True
            )

        pc = PatchCollection(recs, facecolor='none', edgecolor='#CC9966', lw=1, zorder=3)
        ax.add_collection(pc)
        ax.set_xlabel('Time (day)', color = '#CCFFFF', fontsize = 25)
        ax.set_ylabel('Normalized Flux', color = '#CCFFFF', fontsize = 25)
    return ax



In [47]:
def get_epochs(T0, per, min_time, max_time):
    epochs_big = np.array(list(range(-150, 150)))
    new_epochs = T0+(np.array(list(range(-150, 150)))*per)
    
#     print('new epochs', new_epochs)
    epochs = new_epochs[new_epochs>min_time]
#     print('epochs', epochs)
    epochs = epochs[epochs<max_time]
    
    return epochs

In [48]:
toi_700_t0  = [1330.48, 1352.9912, 2303.16, 2349.04]
toi_700_per = [37.4243, 27.8094, 16.0511, 9.97731]

In [49]:
mdwarfs_known

,TICID,RA,DEC,Vmag,Jmag,Tmag,Teff,eTeff_x,Rad,eRad,...,S54,S55,eTeff_y,logg,elogg,M/H,eM/H,GAIA,aLSM,bLSM
0,261257684.0,89.372315,-83.130204,12.588,9.607,10.8460,3751.0,157.0,0.532439,0.015794,...,0.0,0.0,157.0,4.70914,0.009127,NaN,NaN,4.620844e+18,0.2581,0.3609
1,150428135.0,97.096787,-65.579312,13.151,9.469,10.9102,3494.0,157.0,0.419457,0.012573,...,0.0,0.0,157.0,4.80892,0.004651,NaN,NaN,5.284518e+18,0.1604,0.4325
2,219195044.0,92.391740,-53.823484,13.311,10.241,11.5436,3731.0,157.0,0.473141,0.014147,...,0.0,0.0,157.0,4.76101,0.007200,NaN,NaN,5.500474e+18,0.1737,0.4118
3,259377017.0,68.415501,-51.956232,12.603,9.099,10.4981,3532.0,157.0,0.374358,0.011496,...,0.0,0.0,157.0,4.85053,0.002090,NaN,NaN,4.781196e+18,0.1604,0.4325
4,36724087.0,154.646405,-11.716734,13.140,9.007,10.5848,3331.0,157.0,0.374139,0.011184,...,0.0,0.0,157.0,4.85074,0.001634,NaN,NaN,3.767282e+18,0.1529,0.4604
5,235678745.0,285.633044,75.418605,13.190,9.797,11.0788,3746.0,157.0,0.463205,0.013592,...,10.0,10.0,157.0,4.76979,0.006479,NaN,NaN,2.268372e+18,0.1737,0.4118
6,441798995.0,264.147791,77.126122,13.476,10.765,11.9521,3872.0,157.0,0.548484,0.016161,...,10.0,10.0,157.0,4.69525,0.009462,NaN,NaN,1.657159e+18,0.2946,0.3316
7,284441182.0,10.088709,61.213626,13.459,10.154,11.4752,3624.0,157.0,0.490120,0.014531,...,0.0,0.0,157.0,4.74607,0.007723,NaN,NaN,4.272241e+17,0.2125,0.3984
8,467179528.0,197.998168,65.833807,12.941,9.706,11.0402,3618.0,157.0,0.435666,0.012939,...,0.0,0.0,157.0,4.79432,0.005376,NaN,NaN,1.678074e+18,0.1671,0.4201


In [29]:
fits_files = sorted(glob.glob('./known_Mdwarfs_data/PS_data/tic_'+str(150428135)+'/*/*/*'))
fits_files

['./known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2018206045859-s0001-0000000150428135-0120-s',
 './known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2018263035959-s0003-0000000150428135-0123-s',
 './known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2018292075959-s0004-0000000150428135-0124-s',
 './known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2018319095959-s0005-0000000150428135-0125-s',
 './known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2018349182500-s0006-0000000150428135-0126-s',
 './known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2019006130736-s0007-0000000150428135-0131-s',
 './known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2019032160000-s0008-0000000150428135-0136-s',
 './known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2019058134432-s0009-0000000150428135-0139-s',
 './known_Mdwarfs_data/PS_data/tic_150428135/mastDownload/TESS/tess2019085135100-s0010-000000015

In [74]:
column_names = ['TICID', 'planet_name', 'period', 'T0', 'Tdur', 'depth']

# final_file = './search_mdwarf_planets.csv'
new_planet_df = pd.DataFrame(columns=column_names)
print(new_planet_df)


for iii in TICIDs[:1]:
    
    print(iii)
    fits_files = sorted(glob.glob('./known_Mdwarfs_data/FFI_data/all_target_data/tic_'+str(iii)+'_all/*/*/*/*.fits'))
#     fits_files = fits_files
#     print(len(fits_files))
#     fits_files = sorted(glob.glob('./known_Mdwarfs_data/FFI_data/*/*/*/*'+str(iii)+'*/*.fits'))

    count = 0
#     if len(fits_files)>0:
    for file in fits_files:
        sector = np.int64(file.split('/')[7].split('_')[4].split('-')[0][1:])
        print('sector new', sector)

        file = [file]
        count +=1
    # print(tess_spoc_fits)
        print('num files', len(file))
    #     fits_files
        ab, smass, smass_min, smass_max, sradius, sradius_min, sradius_max = get_catalog_info(iii, mdwarfs_known)
        print('val', ab)
        
        total_time, total_flux, total_flux_err = get_data(file)

        time = total_time[np.argsort(total_time)]
        flux = total_flux[np.argsort(total_time)]
        flux_err = total_flux_err[np.argsort(total_time)]
        if np.median(np.diff(time) < 10/60/24):
            print('binned')
            binned_time, binned_flux = new_binning_function_by_time(time, flux, 10)
#         flux_err = np.full(len(flux), np.std(flux))
        
#         indexes = breaking_up_data(new_time, brk)
#         split_times    = [time[indx]     for indx in indexes]
#         split_flux     = [flux[indx]     for indx in indexes]
#         split_flux_err = [flux_err[indx] for indx in indexes]
    
        bboxes = DT_analysis(time, flux, flux_err)
        detrended_lc = make_LightKurveObject(time, flux, flux_err)
    #     print(detrended_lc)

        fig = plt.figure(figsize = (4.7, 9.2))

        ax = fig.add_subplot(111)
        ax.patch.set_facecolor('#CCFFFF') # instead of fig.patch.set_facecolor
        ax.patch.set_alpha(0.65)
        fig.patch.set_facecolor('None') # instead of fig.patch.set_facecolor

        for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)
        for spine in ax.spines.values():
            spine.set_edgecolor('#CCFFFF')
        ax.tick_params(color='#CCFFFF', labelcolor='#CCFFFF', labelsize = 20)
        
        
        plot_lc_with_bboxes(detrended_lc, bboxes, ms=5, marker='.', lw=0, ax = ax)
        plt.subplots_adjust(left=0.35)
        plt.savefig('./images/toi904/new_fig_'+str(iii)+'_s'+str(sector)+'.png')

In [31]:
sectors = [[1], list(range(4,12)), [13], [27,28], [30, 31], list(range(31, 39))]
sectors


[[1],
 [4, 5, 6, 7, 8, 9, 10, 11],
 [13],
 [27, 28],
 [30, 31],
 [31, 32, 33, 34, 35, 36, 37, 38]]

In [32]:
print([len(x)for x in sectors])

[1, 8, 1, 2, 2, 8]


In [33]:
files_indexes = [[0], list(range(1, 1+8)), [9], [10, 11], [12, 13], list(range(14, 14+6))]

In [34]:
files_indexes

[[0],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [9],
 [10, 11],
 [12, 13],
 [14, 15, 16, 17, 18, 19]]

In [35]:

# fits_files

In [36]:
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

flat_sectors = flatten_extend(sectors)

In [49]:
sectors

[[1],
 [4, 5, 6, 7, 8, 9, 10, 11],
 [13],
 [27, 28],
 [30, 31],
 [31, 32, 33, 34, 35, 36, 37, 38]]

In [60]:
T0_2 = 2458714.3512-2457000

p2 = 83.9996 #period

T0_1 = 2459366.628641-2457000

p1 = 10.877 #period

toi904_per = [p1, p2]
toi904_T0  = [T0_1, T0_2]

In [76]:
column_names = ['TICID', 'planet_name', 'period', 'T0', 'Tdur', 'depth']

# final_file = './search_mdwarf_planets.csv'
# new_planet_df = pd.DataFrame(columns=column_names)
# print(new_planet_df)


iii =     261257684
print(iii)
fits_files = sorted(glob.glob('./known_Mdwarfs_data/PS_data/tic_'+str(261257684)+'/*/*/*/*.fits'))
#     fits_files = fits_files
#     print(len(fits_files))
#     fits_files = sorted(glob.glob('./known_Mdwarfs_data/FFI_data/*/*/*/*'+str(iii)+'*/*.fits'))

#     if len(fits_files)>0:fi


# fits_files = [file for file in fits_files if np.int64(file.split('/')[6].split('-')[1].split('-')[0][1:]) in flat_sectors]
# count = 
print('num files', len(fits_files))

for file in fits_files:

    sector = np.int64(file.split('/')[6].split('-')[1].split('-')[0][1:])
#     if sector!=27:
#         continue
    file = [file]

# print(tess_spoc_fits)
    print('num files', len(file))
#     fits_files
    ab, smass, smass_min, smass_max, sradius, sradius_min, sradius_max = get_catalog_info(iii, mdwarfs_known)
    print('val', ab)

    total_time, total_flux, total_flux_err = get_data(file)

    time = np.array(total_time)[np.argsort(total_time)]
    flux = np.array(total_flux)[np.argsort(total_time)]
    flux_err = np.array(total_flux_err)[np.argsort(total_time)]

    intransit = np.logical_or(transit_mask(time, toi_700_per[3],( 2*2.12423/24)+0.1, toi_700_t0[3]), transit_mask(time, toi_700_per[2], ( 2*1.39317/24)+0.1, toi_700_t0[2]))
    
    time = time[~intransit]
    flux = flux[~intransit]
    flux_err = flux_err[~intransit]

#         if np.median(np.diff(time) < 10/60/24):
#             print('binned')
#             binned_time, binned_flux = new_binning_function_by_time(time, flux, 10)
#         flux_err = np.full(len(flux), np.std(flux))

#         indexes = breaking_up_data(new_time, brk)
#         split_times    = [time[indx]     for indx in indexes]
#         split_flux     = [flux[indx]     for indx in indexes]
#         split_flux_err = [flux_err[indx] for indx in indexes]

    bboxes = DT_analysis(time, flux, flux_err)
    detrended_lc = make_LightKurveObject(time, flux, flux_err)
#     print(detrended_lc)

    fig = plt.figure(figsize = (4.7, 9.2))

    ax = fig.add_subplot(111)
    ax.patch.set_facecolor('#CCFFFF') # instead of fig.patch.set_facecolor
    ax.patch.set_alpha(0.8)
    fig.patch.set_facecolor('None') # instead of fig.patch.set_facecolor

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    for spine in ax.spines.values():
        spine.set_edgecolor('#CCFFFF')

    ax.tick_params(color='#CCFFFF', labelcolor='#CCFFFF', labelsize = 20)


    plot_lc_with_bboxes(detrended_lc, bboxes, ms=4, marker='.', lw=0, ax = ax)
    ymin, ymax = ax.get_ylim()

    for jjj in range(len(toi904_per)):
        epochs = get_epochs(toi904_T0[jjj], toi904_per[jjj], min(time), max(time))
#             print('len epochs', len(epochs))
        ax.vlines(epochs, ymin, ymax, color = 'C'+str(jjj+10), zorder = -1)

    plt.subplots_adjust(left=0.35)
    plt.savefig('./images/toi904/PS/new_fig_s'+str(sector)+'.png')
#     count+=1


In [70]:
column_names = ['TICID', 'planet_name', 'period', 'T0', 'Tdur', 'depth']

# final_file = './search_mdwarf_planets.csv'
new_planet_df = pd.DataFrame(columns=column_names)
print(new_planet_df)


iii =     150428135
print(iii)



print(iii)
fits_files = sorted(glob.glob('./known_Mdwarfs_data/PS_data/tic_'+str(150428135)+'/*/*/*/*.fits'))

#     fits_files = fits_files
#     print(len(fits_files))
#     fits_files = sorted(glob.glob('./known_Mdwarfs_data/FFI_data/*/*/*/*'+str(iii)+'*/*.fits'))

#     if len(fits_files)>0:

# fits_files = [file for file in fits_files if np.int64(file.split('/')[6].split('-')[1].split('-')[0][1:]) in flat_sectors]

# subset_files = [np.array(fits_files)[np.array(index)] for index in files_indexes]

# fits_files = sorted(glob.glob('./known_Mdwarfs_data/PS_data/tic_'+str(150428135)+'/*/*/*/*.fits'))
#     fits_files = fits_files
#     print(len(fits_files))
#     fits_files = sorted(glob.glob('./known_Mdwarfs_data/FFI_data/*/*/*/*'+str(iii)+'*/*.fits'))

#     if len(fits_files)>0:


# subset_files = [np.array(fits_files)[np.array(index)] for index in  files_indexes]
count = 0

for files in fits_files:

# print(tess_spoc_fits)
    print('num files', len(file))
#     fits_files
    ab, smass, smass_min, smass_max, sradius, sradius_min, sradius_max = get_catalog_info(iii, mdwarfs_known)
    print('val', ab)

    total_time, total_flux, total_flux_err = get_data(files)

    time = np.array(total_time)[np.argsort(total_time)]
    flux = np.array(total_flux)[np.argsort(total_time)]
    flux_err = np.array(total_flux_err)[np.argsort(total_time)]
    
    intransit = np.logical_or(transit_mask(time, toi_700_per[3],( 2*2.12423/24)+0.1, toi_700_t0[3]), transit_mask(time, toi_700_per[2], ( 2*1.39317/24)+0.1, toi_700_t0[2]))


    time = time[~intransit]
    flux = flux[~intransit]
    flux_err = flux_err[~intransit]
    
#         if np.median(np.diff(time) < 10/60/24):
#             print('binned')
#             binned_time, binned_flux = new_binning_function_by_time(time, flux, 10)
#         flux_err = np.full(len(flux), np.std(flux))

#         indexes = breaking_up_data(new_time, brk)
#         split_times    = [time[indx]     for indx in indexes]
#         split_flux     = [flux[indx]     for indx in indexes]
#         split_flux_err = [flux_err[indx] for indx in indexes]

    bboxes = DT_analysis(time, flux, flux_err)
    detrended_lc = make_LightKurveObject(time, flux, flux_err)
#     print(detrended_lc)

    fig = plt.figure(figsize = (50, 10))

    ax = fig.add_subplot(111)
    ax.patch.set_facecolor('#CCFFFF') # instead of fig.patch.set_facecolor
    ax.patch.set_alpha(0.65)
    fig.patch.set_facecolor('None') # instead of fig.patch.set_facecolor

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    for spine in ax.spines.values():
        spine.set_edgecolor('#CCFFFF')

    ax.tick_params(color='#CCFFFF', labelcolor='#CCFFFF', labelsize = 20)


    plot_lc_with_bboxes(detrended_lc, bboxes, ms=5, marker='.', lw=0, ax = ax)
    ymin, ymax = ax.get_ylim()

    for jjj in range(1):
        epochs = get_epochs(toi_700_t0[jjj], toi_700_per[jjj], min(time), max(time))
#             print('len epochs', len(epochs))
        ax.vlines(epochs, ymin, ymax, color = 'C'+str(jjj+3))

    plt.subplots_adjust(left=0.35)
    plt.savefig('./images/toi904/PS/new_fig_chunk'+str(count)+'.png')
    count+=1


IsADirectoryError: [Errno 21] Is a directory: '.'

In [25]:
time_oot = np.array(pd.read_csv('toi700_oot.csv')['time'])
flux_oot = np.array(pd.read_csv('toi700_oot.csv')['flux'])
flux_err_oot = np.array(pd.read_csv('toi700_oot.csv')['flux_err'])


In [26]:
time_oot

array([1325.65662662, 1325.67745988, 1325.71912641, ..., 2360.3717479 ,
       2360.3786923 , 2360.38563669])

In [27]:
np.array(pd.read_csv("indexes"+str(1)+".csv", header = None).astype(int))[:,0]


array([ 993,  994,  995, ..., 7923, 7924, 7925])

In [42]:
indexes_split = []

for i in range(6):
    ary = np.array(pd.read_csv("indexes"+str(i)+".csv", header = None).astype(int))[:,0]
    
    indexes_split.append(ary)


max_val = 0
for lst in indexes_split:
    max_lst = max(lst)
    if max_lst>max_val:
        max_val = max_lst

In [43]:
[val[0] for val in indexes_split]

np.argsort([val[0] for val in indexes_split])
indexes_split = np.array(indexes_split)[np.argsort([val[0] for val in indexes_split])]

In [44]:
max_val

38042

In [45]:
toi_700_t0  = [1330.48, 1352.9912, 2303.16, 2349.04]
toi_700_per = [37.4243, 27.8094, 16.0511, 9.97731]

In [46]:
import matplotlib.ticker as ticker

In [51]:
column_names = ['TICID', 'planet_name', 'period', 'T0', 'Tdur', 'depth']

# final_file = './search_mdwarf_planets.csv'
new_planet_df = pd.DataFrame(columns=column_names)
print(new_planet_df)


for iii in TICIDs[:1]:
    
    print(iii)
    fits_files = sorted(glob.glob('./known_Mdwarfs_data/FFI_data/all_target_data/tic_'+str(iii)+'_all/*/*/*/*.fits'))
#     fits_files = fits_files
#     print(len(fits_files))
#     fits_files = sorted(glob.glob('./known_Mdwarfs_data/FFI_data/*/*/*/*'+str(iii)+'*/*.fits'))

#     if len(fits_files)>0:
    intransit_index = 0
    ab, smass, smass_min, smass_max, sradius, sradius_min, sradius_max = get_catalog_info(iii, mdwarfs_known)
    print('val', ab)
    get_data(fits_files)
    total_time = []
    total_flux = []
    total_flux_err = []
    
    for i in fits_files:
        sector = np.int64(i.split('/')[7].split('_')[4].split('-')[0][1:])
        print('sector', sector)
#         if sector > 34:
#             print(hdulist)

        hdulist=apf.open(i) #fits time series

        tbdata = hdulist[1].data
#         print(tbdata)
        qual         = tbdata.QUALITY == 0

        time, flux, flux_raw, flux_err = tbdata.TIME[qual], tbdata.KSPSAP_FLUX[qual],  tbdata.SAP_FLUX[qual],tbdata.KSPSAP_FLUX_ERR[qual]

        flux_err = flux_err/np.nanmedian(flux)
        flux = flux/np.nanmedian(flux)
        flux, flux_err, time = flux[~np.isnan(flux)], flux_err[~np.isnan(flux)],  time[~np.isnan(flux)]
        
        outliers_mask = mask_outliers(time, flux_raw)
        
        time = time[~outliers_mask]
        flux = flux[~outliers_mask]
        flux_err = flux_err[~outliers_mask]

        total_time.extend(time)
        total_flux.extend(flux)
        total_flux_err.extend(flux_err)


    total_time = np.array(total_time)
    total_flux = np.array(total_flux)
    total_flux_err = np.array(total_flux_err)
    
    time = total_time[np.argsort(total_time)]
    flux = total_flux[np.argsort(total_time)]
    flux_err = total_flux_err[np.argsort(total_time)]
    
    print(len(time))
    
    print('time', len(time))

#         print('blah', sec_oot)
#     if np.median(np.diff(time) < 30/60/24):
#         print('binned')
#         binned_time, binned_flux = new_binning_function_by_time(time, flux, 30)
# #         flux_err = np.full(len(flux), np.std(flux))


    for iii in range(len(indexes_split)):
        
        new_time = time[indexes_split[iii]]
        new_flux = flux[indexes_split[iii]]
        new_flux_err = flux_err[indexes_split[iii]]

#         indexes = breaking_up_data(new_time, brk)
#         split_times    = [time[indx]     for indx in indexes]
#         split_flux     = [flux[indx]     for indx in indexes]
#         split_flux_err = [flux_err[indx] for indx in indexes]

        bboxes = DT_analysis(new_time, new_flux, new_flux_err, is_flat = False)
        detrended_lc = make_LightKurveObject(new_time, new_flux, new_flux_err)
    #         print(detrended_lc)

        fig = plt.figure(figsize = (100, 10))

        ax = fig.add_subplot(111)
        ax.patch.set_facecolor('#CCFFFF') # instead of fig.patch.set_facecolor
        ax.patch.set_alpha(0.65)
        
        
        ax.xaxis.set_major_locator(ticker.MultipleLocator(25)) 
        for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)
#         for spine in ax.spines.values():
#             spine.set_edgecolor('#CCFFFF')
#         ax.tick_params(color='#CCFFFF', labelcolor='#CCFFFF')
        ax.tick_params(color='k', labelcolor='k')

        plot_lc_with_bboxes(detrended_lc, bboxes, ms=3, marker='.', lw=0, ax = ax)
        
        ymin, ymax = ax.get_ylim()
        
#         print('times edges', min(new_time), max(new_time), abs(min(new_time)-max(new_time)))
        for jjj in range(1):
            epochs = get_epochs(toi_700_t0[jjj], toi_700_per[jjj], min(new_time), max(new_time))
#             print('len epochs', len(epochs))
            ax.vlines(epochs, ymin, ymax, color = 'C'+str(jjj+3))
        
        print('good')
        plt.show()
        plt.savefig('./images/toi700/raw/new_fig_chunk_'+str(iii)+'_oot_raw.png')

Empty DataFrame
Columns: [TICID, planet_name, period, T0, Tdur, depth]
Index: []
150428135
val [0.1604 0.4325]
init time 993
init time 701
init time 1034
init time 829
init time 1029
init time 673
init time 1001
init time 950
init time 792
init time 1120
init time 3114
init time 3019
init time 2868
init time 3047
init time 3439
init time 3113
init time 2108
init time 2493
init time 2871
init time 2927
total_time 38043
sector 1
sector 4
sector 5
sector 6
sector 7
sector 8
sector 9
sector 10
sector 11
sector 13
sector 27
sector 28
sector 30
sector 31
sector 33
sector 34
sector 35
sector 36
sector 37
sector 38
38043
time 38043
good
good
good
good
good
good


In [54]:
sectors = [[1], list(range(4,12/), [13], [27,28], [30, 31], list(range(31, 39))]

In [55]:
sectors

[[1],
 [4, 5, 6, 7, 8, 9, 10, 11],
 [13],
 [27, 28],
 [30, 31],
 [31, 32, 33, 34, 35, 36, 37, 38]]

In [35]:
new_indexes_split = [range(min(x), max(x)) for x in indexes_split]

In [36]:

axis_num = 0

indexes_split

split_times = [time[indx] for indx in indexes_split]
split_flux = [flux[indx] for indx in indexes_split]

sec_diff = [max(time_sec)-min(time_sec) for time_sec in split_times] 

ratios = [diff/min(sec_diff) for diff in sec_diff]

ratios

[1.0,
 7.75192618968921,
 1.009861206342858,
 1.7660585175735324,
 1.96030495662672,
 5.775430183463783]

In [57]:
def creating_multi_sector_plot(time, flux, indexes_split):
    plt.rcParams.update({
    "font.weight": "normal",
    "xtick.major.size": 20,
    "xtick.major.pad": 0.01,
    "xtick.labelsize": 35,
    "ytick.major.size": 20,
    "ytick.major.pad": 1,
    "ytick.labelcolor": '#CCFFFF',
    "xtick.labelcolor": '#CCFFFF',
    
    "grid.color": '#CCFFFF',
    "grid.linestyle": "-",
    "grid.linewidth": 15,
    "lines.linewidth": 2,
    "lines.color": '#CCFFFF'})
    

    split_times    = [time[indx] for indx in indexes_split]
    split_flux     = [flux[indx] for indx in indexes_split]

    sector_time_differences = [max(time_chunk)-min(time_chunk) for time_chunk in split_times]

    diff_time_min = min(sector_time_differences)
    
    sector_ratios = [(diff_time_sector+3)/(diff_time_min+3) for diff_time_sector in sector_time_differences]

    
    fig, axes = plt.subplots(1, len(indexes_split), figsize = [75, 10], sharey=True, 
                             gridspec_kw={'width_ratios': sector_ratios})

    fig.patch.set_facecolor('None') # instead of fig.patch.set_facecolor
#     fig.patch.set_alpha(0.8)
    
    d = .03 # how big to make the diagonal lines in axes coordinates

    for jjj in range(len(indexes_split)):
        
        sec_time = split_times[jjj]
        sec_flux = split_flux[jjj]
        axes[jjj].patch.set_facecolor('#0099CC') # instead of fig.patch.set_facecolor
        axes[jjj].patch.set_alpha(0.4)
        axes[jjj].tick_params(color = '#CCFFFF', which='both', width = 3)
        axes[jjj].tick_params(which='major', length = 14)
        axes[jjj].tick_params(which='minor', length = 8)

#         axes[jjj].scatter(sec_time, sec_flux, color = 'lightgrey', s = 50)
        axes[jjj].scatter(sec_time, sec_flux, color = '#CC9966', linewidth = 5)

        axes[jjj].set_xlim(min(sec_time)-1.5, max(sec_time)+1.5)
        axes[jjj].spines['bottom'].set_color('#CCFFFF')
        axes[jjj].spines['top'].set_color('#CCFFFF')
        
        epochs = get_epochs(toi_700_t0[2], toi_700_per[2], min(sec_time)-1.5, 1.5+max(sec_time))
#             print('len epochs', len(epochs))

        axes[jjj].vlines(np.array(epochs), ymin, ymax, color = 'k')

        axes[0].spines['left'].set_color('#CCFFFF')
        
        axes[5].spines['right'].set_color('#CCFFFF')

        kwargs = dict(transform=axes[jjj].transAxes, color='#CCFFFF', clip_on=False)
    

        if jjj<max(range(len(indexes_split))):
            axes[jjj].spines['right'].set_visible(False)
            axes[jjj+1].spines['left'].set_visible(False)
            

            axes[jjj].plot((1-(d/sector_ratios[jjj]),1+(d/sector_ratios[jjj])),(-d,+d), **kwargs) # top-left diagonal
            axes[jjj].plot((1-(d/sector_ratios[jjj]),1+(d/sector_ratios[jjj])),(1-d,1+d), **kwargs) # bottom-left diagonal


        if jjj>0:
            axes[jjj].yaxis.set_ticks_position('none')

            axes[jjj].plot((-d/sector_ratios[jjj],d/sector_ratios[jjj]),(-d,+d), **kwargs) # top-right diagonal
            axes[jjj].plot((-d/sector_ratios[jjj],d/sector_ratios[jjj]),(1-d,1+d), **kwargs) # bottom-right diagonalaxes[1].plot((1-d,1+d), (-d,+d), **kwargs)
        
#             kwargs.update(transform=axes[jjj].transAxes) # switch to the bottom axes
#             axes[jjj].plot((-d/sector_ratios[jjj],d/sector_ratios[jjj]),(-d/sector_ratios[jjj],+d/sector_ratios[jjj]), **kwargs) # top-right diagonal
#             axes[jjj].plot((-d/sector_ratios[jjj],d/sector_ratios[jjj]),(1-d/sector_ratios[jjj],1+d/sector_ratios[jjj]), **kwargs) # bottom-right diagonalaxes[1].plot((1-d,1+d), (-d,+d), **kwargs)


    # plt.subplots_adjust(wspace=0, hspace=0.23)
    # plt.xlabel('Time (BJD - 2457000)', fontsize = 45)
    fig.text(0.441, 0.05, 'Time (BJD - 2457000)', va='center', fontsize = 45,  fontweight = 'normal', color ='#CCFFFF')

    fig.text(0.085, 0.5, 'Relative Flux', va='center', fontsize = 45,  rotation='vertical', fontweight = 'normal', color = '#CCFFFF')
    
    plt.subplots_adjust(bottom=0.2)
    
    fig.savefig('evil_2.png', facecolor = fig.get_facecolor(), edgecolor = 'none')
    
creating_multi_sector_plot(time, flux, new_indexes_split)